In [1]:
import os

import json
import random
from collections import Counter

from konlpy.tag import Komoran
komoran = Komoran()

os.chdir('../')
import yake
from yake.korea_token import edit_josa, edit_sentences
os.chdir('test_dacon/')

from sklearn.feature_extraction.text import TfidfVectorizer 
from collections import defaultdict

from krwordrank.word import KRWordRank
import math

from sentence_transformers import SentenceTransformer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from pandas import DataFrame
from test_testmain import RougeScorer

import warnings
warnings.filterwarnings(action='ignore')

/Users/bigtree/Desktop/During_Bigtree/Kyake/.env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("valid_original.json", "r", encoding="utf8") as f:
    contents = f.read() # string 타입
    json_data = json.loads(contents)
    f.close()

news_list = []
for _ in range(300):
    news_list.append(random.randint(0, 30000))  # max: 30122 || 30122개의 뉴스 데이터

news_set = set(news_list)
news_list = list(news_set)

In [4]:
def noun_parser(txt:str):
    noun = komoran.nouns(txt)
    count = Counter(noun)

    # 명사 빈도 카운트q
    noun_list = count.most_common(10)
    return_list = []
    for i in noun_list:
        return_list.append(i[0])

    return return_list

In [5]:
def tfidf_parser(txt:list):
    vectorizer = TfidfVectorizer()
    vectorizer.fit(txt)
    matrix = vectorizer.transform(txt)

    vocabulary_word_id = defaultdict(int)
        
    for idx, token in enumerate(vectorizer.get_feature_names()):
        vocabulary_word_id[token] = idx
        
    # 특징 추출 결과: {"token": value}
    result = defaultdict(str)
        
    for token in vectorizer.get_feature_names():
        result[token] = matrix[0, vocabulary_word_id[token]]
        
    # 내림차순 (중요도 high) 기준 정렬
    result = sorted(result.items(), key = lambda item: item[1], reverse = True)
    return_list = []
    for i in result:
        if i[1] == 0:
            continue
        else:
            if len(return_list) < 10:
                return_list.append(i[0])
            else:
                continue
    return return_list

In [6]:
def wordrank_parser(txt:list):
    wordrank_extractor = KRWordRank(min_count=5, max_length=10)
    keywords, rank, graph = wordrank_extractor.extract(txt, num_keywords=100)

    def make_vocab_score(keywords, scaling=None):
        if scaling is None:
            scaling = lambda x:math.sqrt(x)
        return {word:scaling(rank) for word, rank in keywords.items()}

    keywords = make_vocab_score(keywords)
    return_list = []
    for i in keywords:
        if len(return_list) < 10:
            return_list.append(i[0])
        else:
            continue
    
    return return_list

In [7]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

def keybert_parser(txt:list, model):
    
    tokenized_doc = komoran.pos(txt)
    tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] in ['NNG','NNP']])

    # print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
    # print('명사 추출 :',tokenized_nouns)

    n_gram_range = (1, 3)
    count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
    candidates = count.get_feature_names_out()

    # print('trigram 개수: ', len(candidates))
    # print('trigram 5개 출력: ', candidates[:5])


    doc_embedding = model.encode([txt])
    candidate_embeddings = model.encode(candidates)

    top_n = 10
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

    return keywords

In [8]:
def kyake(txt:list):
    return_list = []

    kw_extractor = yake.KeywordExtractor(n=[1,2,3],top=10, stoplen=2, windowsSize=1, WG=False, ReDup=False)

    return_tuple = kw_extractor.extract_keywords(''.join(txt))
    
    for i in return_tuple:
        return_list.append(i[0])

    return return_list

In [21]:
df_trainset = DataFrame(columns=["id", "summary"])
df_testset = DataFrame(columns=["id", "summary"])

scorer = RougeScorer()
_dataframe = DataFrame()

R1 = []
R2 = []
Rl = []

for i in news_list:
    
    id = None
    text_list = []
    topic_number = None

    for j in range(0,len(json_data["documents"][i]["text"]),1):
        for k in range(0,len(json_data["documents"][i]["text"][j]),1):
            # print(json_data["documents"][i]["text"][j][f]["sentence"])
            elements = json_data["documents"][i]["text"][j][k]["sentence"] + '.'
            text_list.append(elements)

    topic_number = json_data["documents"][i]["extractive"]
    id = [int(json_data["documents"][0]['id'])]

    text = [' '.join(text_list)]
    STRTexT = ' '.join(text_list)
    LISTText = text_list

    new_text = edit_sentences(STRTexT)
    # print('원본 split_sentences :', new_text)
    total_value = []
    for x in range(len(new_text)):
        total_value.append(' '.join(edit_josa(new_text[x])))
    TUNNEDText = total_value

    ### Kyake 키워드 추출: kyake(text)                     || txt:list

    ### 빈도수 기반 명사 추출: noun_parser(STRTexT)      	 ||	txt:str

    ### TF.IDF: tfidf_parser(TUNNEDText)    	        ||	txt:list

    ### KR-WordRank: wordrank_parser(TUNNEDText)	    ||	txt:list

    ### Korean KeyBERT" keybert_parser(STRTexT, model) 	||	txt:list

    """
    return : list
    """
    
    # keywords = kyake(text)
    # keywords = noun_parser(STRTexT)
    keywords = tfidf_parser(TUNNEDText)
    # keywords = wordrank_parser(TUNNEDText)
    # keywords = keybert_parser(STRTexT, model)
    
    score = 0
    for l in range(len(text_list)):     # text_list[l] : text_list의 l번째 문장
        for m in keywords:              # keywords : text파일의 문서를 각 함수들을 돌린 후 도출되는 키워드들.
            if m in text_list[l]:
                score = score + 1*(1+0.2*len(m[0].split()))
        score_data = dict(zip(["id", "index", "score"], [i, l, score]))
        score = 0
        _dataframe = _dataframe.append(score_data, ignore_index=True)

    multi_index = _dataframe.sort_values(by=['score', 'index'], ascending=[False, True]).groupby("id").head(3)
    # print(multi_index)
    multi_index = multi_index.sort_values(by=['id','score'], ascending=[True, False])
    # print(multi_index)
    # print(multi_index[multi_index['id']==i])

    summary_number = [int (i) for i in list(multi_index[multi_index['id']==i]['index'])]

    df_testset['id'] = id
    df_testset['summary'] = [[text_list[int(summary_number[0])],text_list[int(summary_number[1])], text_list[int(summary_number[2])]]]
    df_trainset['id'] = id
    df_trainset['summary'] = [[text_list[int(topic_number[0])],text_list[int(topic_number[1])], text_list[int(topic_number[2])]]]

    score_set = scorer.compute_rouge(df_trainset, df_testset)
    rouge_1 = score_set[0]
    rouge_2 = score_set[1]
    rouge_l = score_set[2]

    R1.append(rouge_1)
    R2.append(rouge_2)
    Rl.append(rouge_l)

    print(i,"번째 문단..")
    print("내가 선택한 문장은..? : ", (summary_number))
    print([text_list[int(summary_number[0])],text_list[int(summary_number[1])], text_list[int(summary_number[2])]])
    print("정답지는..! : ", topic_number)
    print([text_list[int(topic_number[0])],text_list[int(topic_number[1])], text_list[int(topic_number[2])]])
    print()

print("rouge_1의 평균점수: ", sum(R1) / len(R1))
print("rouge_2의 평균점수: ", sum(R2) / len(R2))
print("rouge_l의 평균점수: ", sum(Rl) / len(Rl))

7176 번째 문단..
내가 선택한 문장은..? :  [0, 1, 5]
['박미경 한국여성벤처협회 회장이 국내외 위기상황을 벤처기업 간 협업으로 돌파하자고 강조했다..', '박 회장은 30일 막을 내린 벤처썸머포럼 폐회사를 통해 "특별한 자원이 없는 대한민국이 성장하려면 벤처기업 육성에 집중해야 한다"며 "벤처기업도 이 같은 기회를 잘 활용해야 한다"고 밝혔다..', '"서로 응원하고 동반성장을 꾀하는 벤처기업 문화를 통해 함께 성장하길 바란다"며 "이 문화를 바탕으로 벤처 생태계가 활성화될 것"이라고 박 회장은 기대했다..']
정답지는..! :  [1, 7, 9]
['박 회장은 30일 막을 내린 벤처썸머포럼 폐회사를 통해 "특별한 자원이 없는 대한민국이 성장하려면 벤처기업 육성에 집중해야 한다"며 "벤처기업도 이 같은 기회를 잘 활용해야 한다"고 밝혔다..', '박 회장은 "기업가 정신과 혁신 전략, 규제 혁파를 통해 벤처기업 성장 기반을 마련해야 한다"며 "자금과 정부 관심이 벤처 생태계에 쏠리는 만큼 지금이 제2 벤처 붐을 이끌 적기"라고 설명했다..', '벤처기업이 모여 경영전략과 정보를 공유하는 네트워크 행사다..']

5641 번째 문단..
내가 선택한 문장은..? :  [0, 9, 8]
["롯데마트가 오는 8일부터 14일까지 일주일간 '피마늘(1개)'을 100원에 판매한다..", "해당 행사 기간 동안 롯데마트는 '피마늘(1개)'을 단돈 100원에 판매한다..", '이처럼 마늘 농가 상황이 악화되자 롯데마트에서는 오는 8일부터 일주일간 전점에서 마늘 소비 촉진 행사에 나선다..']
정답지는..! :  [0, 1, 8]
["롯데마트가 오는 8일부터 14일까지 일주일간 '피마늘(1개)'을 100원에 판매한다..", '이번 행사는 마늘 가격 폭락에 힘겨워 하는 농가의 재고 소진을 도와 마늘 가격 정상화를 위해 기획됐다..', '이처럼 마늘 농가 상황이 악화되자 롯데마트에서는 오는 8일부터 일주일간 전점에서 마늘 소비 촉진 행사에 나선다..']

1383